# 갱신하고자 하는 기업의 이름, 시장구분, 업종명을 아래 리스트에 채워주세요

# path

In [11]:
import os

# 파일이 있는 현재 위치를 path로 지정
current_directory = os.getcwd()
path = current_directory.replace('\\', '/')
path += '/' if not path.endswith('/') else ''

data_path = path + "data/"
print("Data_path :", data_path)

img_path = path + "img/"
print("IMG_path :", img_path)

#필요시 path 재조정
# path = 'C:/Users/sook7/미래에솦_주피터/본선코드정리/'
# data_path = path + "data/"
# img_path = path + "img/"

save_to = path + 'streamlit_mockup/data'

Data_path : C:/Users/sook7/미래에솦_주피터/[빅데이터_미래에솦]소스코드/data/
IMG_path : C:/Users/sook7/미래에솦_주피터/[빅데이터_미래에솦]소스코드/img/


# 예측했던 기업이 상장한 뒤 정보가 갱신되면 크롤링하여 시초/공모(%)값과 업황을 채워 넣는다.

In [12]:
#패키지 호출

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

import requests
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.parse import urlparse
from pandas.io.html import read_html
import time
import sys
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.

# 업종 목록
 
### 코스피
'코스피', '음식료품', '섬유의복', '종이목재', '화학', '의약품', '비금속광물', '철강금속', '기계',
       '전기전자', '의료정밀', '운수장비', '유통업', '전기가스업', '건설업', '운수창고업', '통신업', '금융업',
       '증권', '보험', '서비스업', '제조업'

### 코스닥
'코스닥', '화학', '종이·목재', '제약', '일반전기전자', '의료·정밀기기', '음식료·담배',
       '운송장비·부품', '섬유·의류', '비금속', '기계·장비', '금속', 'IT H/W', 'IT S/W & SVC',
       '통신방송서비스', '유통', '운송', '오락·문화', '금융', '건설', '제조업', 'IT부품', '반도체',
       '정보기기', '통신장비', '컴퓨터서비스', '소프트웨어', '디지털컨텐츠', '인터넷', '방송서비스', '통신서비스',
       '기타제조', '출판·매체복제', '기타서비스'

In [13]:
#갱신할 기업 목록, 업종
company_list = ['시지트로닉스']
market_list = ['KOSDAQ']
# 실제로는 반도체기업, 업황이 바뀌는 걸 확인해보기 위한 예시. 원래 업황값은 3.03, 1.94로 성공적으로 바뀐것을 확인할 수 있음
sector_list = ['IT H/W'] 


In [14]:
def crawling_38(company_list):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.implicitly_wait(2) 
    driver.get('http://www.38.co.kr/html/fund/index.htm?o=r1')

    df_df = pd.DataFrame()
    df_ls = pd.DataFrame()

    for company_name in company_list:
        for page in range(1, 4):  # 최대 3페이지까지 수집
            ### 수요예측결과 페이지
            url = f"https://www.38.co.kr/html/fund/index.htm?o=r1&page={page}"
            response = requests.get(url)
            response.raise_for_status()

            # HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')

            # 테이블 선택 및 파싱
            table = soup.find('table', summary="수요예측결과")
            headers = [header.get_text().strip() for header in table.find_all('th')]
            rows = [[col.get_text().strip() for col in row.find_all('td')] for row in table.find_all('tr')]

            # DataFrame 생성 및 추가
            df_df_temp = pd.DataFrame(rows[2:], columns=headers)  # 첫 번째 행은 헤더와 동일하므로 제거
            if company_name in df_df_temp['기업명'].values:
                df_df_temp = df_df_temp[df_df_temp['기업명'].str.contains(company_name)]
                df_df_temp['기업명'] = company_name 
                
                # 종목코드 가져오기
                target_text = company_name
                element = None
                for link in soup.find_all('a'):
                    if target_text in link.text:
                        element = link
                        break

                if element:
                    # 해당 링크로 이동
                    link = 'http://www.38.co.kr/html/fund/' + element.attrs['href']
                    if link:
                        driver.get(link)
                        종목코드 = driver.find_element(By.XPATH, "/html/body/table[3]/tbody/tr/td/table[1]/tbody/tr/td[1]/table[2]/tbody/tr[2]/td[4]")
                        종목코드 = 종목코드.text.strip()
                        driver.back()
                    else:
                        print("링크를 찾을 수 없습니다.")
                        break
                else:
                    print("텍스트를 찾을 수 없습니다.")
                    break

                df_df_temp['종목코드'] = 종목코드
                break
                
            else:
                print(f'수요예측결과 {page} 페이지에는 {company_name}이 없습니다.')
            
        #company_name에 대한 기업정보 저장
        df_df = pd.concat([df_df, df_df_temp])
            
            ### 신규상장 페이지
        
        for page in range(1, 4):  # 최대 3페이지까지 수집
            ### 수요예측결과 페이지
            url = f"https://www.38.co.kr/html/fund/index.htm?o=nw&page={page}"
            response = requests.get(url)
            response.raise_for_status()

            # HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')

            # 테이블 선택 및 파싱
            table = soup.find('table', summary="신규상장종목")
            headers = [header.get_text().strip() for header in table.find_all('th')]
            rows = [[col.get_text().strip() for col in row.find_all('td')] for row in table.find_all('tr')]

            # DataFrame 생성 및 추가
            df_ls_temp = pd.DataFrame(rows[2:], columns=headers)  # 첫 번째 행은 헤더와 동일하므로 제거
            if company_name in df_ls_temp['기업명'].values:
                df_ls_temp = df_ls_temp[df_ls_temp['기업명'].str.contains(company_name)]
                df_ls_temp['기업명'] = company_name 
                break
            
            else:
                print(f'신규상장 {page} 페이지에는 {company_name}이 없습니다.')
                        
        #company_name에 대한 기업정보 저장
        df_ls = pd.concat([df_ls, df_ls_temp])
            
        # 서버에 부담을 주지 않기 위해 잠시 대기
        time.sleep(1)
    
    df = pd.merge(df_df, df_ls.drop('공모가(원)', axis=1), on='기업명', how='left')
    df = df[['기업명','종목코드','예측일','시초/공모(%)']]

   
    driver.quit()
    return df

In [15]:
df = crawling_38(company_list)
df

신규상장 1 페이지에는 시지트로닉스이 없습니다.


,기업명,종목코드,예측일,시초/공모(%)
0,시지트로닉스,429270,2023.07.18,5.4%


# 업종명 수기로 채워넣은뒤 업황 갱신

In [16]:
def renew_sector(market_list, sector_list, df):
    df_ksp = pd.read_csv(data_path + 'data_KOSPI업황_update.csv')
    df_kdq = pd.read_csv(data_path + 'data_KOSDAQ업황_update.csv')
    
    df_ksp['일자'] = pd.to_datetime(df_ksp['일자'])
    df_kdq['일자'] = pd.to_datetime(df_kdq['일자'])
    df['예측일'] = pd.to_datetime(df['예측일'])
    
    i = 0
    
    for market, sector in zip(market_list, sector_list):
        
        date = df.loc[i,'예측일']
        
        if market == 'KOSPI':
            data = df_ksp[df_ksp['일자'] == date][sector].values[0]
        
        elif market == 'KOSDAQ':
            data = df_kdq[df_kdq['일자'] == date][sector].values[0]
        
        df.at[i, '업황'] = data
        i += 1
    
    return df

In [17]:
df = renew_sector(market_list, sector_list, df)
df['종목코드'] = df['종목코드'].astype(str).str.zfill(6)

In [18]:
df

,기업명,종목코드,예측일,시초/공모(%),업황
0,시지트로닉스,429270,2023-07-18,5.4%,1.943244


# 업데이트
### 예측했던 기업이 상장해서 시초/공모(%)의 결과와 업종이 명확해 졌으면 해당 정보로 데이테프레임을 업데이트한다.
- data_전처리_model_final.csv의 업황을 올바르게 업데이트
- data_streamlit_df.csv의 시초/공모(%)를 올바르게 업데이트


In [19]:
# load data_전처리_model_final.csv
model_df = pd.read_csv(data_path+'data_전처리_model_final.csv')
model_df['종목코드'] = model_df['종목코드'].astype(str).str.zfill(6)

for i, code in enumerate(df['종목코드'].values):
    model_df.loc[model_df['종목코드'] == code, '업황'] = df.loc[i, '업황']
    
model_df
    
# model_df.to_csv(data_path + 'data_전처리_model_final.csv',index=False)
# model_df.to_csv(save_to + 'data_전처리_model_final.csv',index=False)

,종목코드,종목명,매출액 증감율,EPS 증감율,자산비율,유동비율,예상 시가총액,PSR,ROE,nsi_5,업황,공모주 시장 동향,유통가능물량비율,기관경쟁률,의무보유확약비율,범주,지수명,종목구분
0,429270,시지트로닉스,-0.777685,-0.316154,-0.028382,-0.600313,-0.420999,-0.781777,-1.747224,0.601365,1.943244,0.586120,1.287495,2.018226,-0.277130,3.0,반도체,KOSDAQ
1,373170,엠아이큐브솔루션,-0.182593,1.863832,0.002239,-0.331048,-0.722887,-0.781777,1.166004,0.526371,-0.996045,0.586120,-1.213963,2.349645,1.377084,2.0,소프트웨어,KOSDAQ
2,440110,파두,1.078251,-2.550547,-0.374064,1.544702,1.120884,-0.781775,-14.474059,0.084439,-3.613017,0.586120,-0.001359,-0.384109,0.871591,0.0,반도체,KOSDAQ
3,448710,코츠테크놀로지,-0.177669,-0.423642,-0.039756,-0.407834,-0.678235,-0.781778,-0.289018,-0.113091,-3.232645,0.586120,-0.838633,2.256489,0.546428,2.0,IT H/W,KOSDAQ
4,445680,큐리옥스바이오시스템즈,-0.179530,-0.324929,-0.769584,0.959710,-0.468003,-0.781764,-1.489582,-0.113091,1.580788,0.586120,-0.169406,-0.690447,-0.596613,1.0,의료·정밀기기,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,131220,대한과학,-0.397191,-0.336389,0.173354,-0.463176,-0.951454,-0.709857,0.176053,-0.798978,-1.299180,1.745729,0.427272,-0.753148,-0.681809,2.0,유통,KOSDAQ
759,110570,넥솔론,1.240459,0.857775,1.489979,-0.838836,0.730689,-0.695716,0.065718,-1.705831,2.824264,1.745729,-0.777189,-1.026899,-0.681809,1.0,NaN,NaN
760,123330,제닉,0.849816,0.529857,1.010636,-0.717767,-0.566488,-0.654357,1.239848,1.028566,0.538425,0.640783,1.344498,-0.737025,-0.681809,2.0,제조업,KOSDAQ
761,040910,아이씨디,2.773318,1.484697,1.480825,-0.700771,0.406509,0.113108,0.236469,1.028566,-0.377435,0.640783,1.910824,-0.736935,-0.681809,2.0,IT H/W,KOSDAQ


### 시초/공모(%) 변경

In [20]:
# load sttreamlit_df.csv
streamlit = pd.read_csv(data_path+'data_streamlit_df.csv')
streamlit['종목코드'] = streamlit['종목코드'].astype(str).str.zfill(6)

for i, code in enumerate(df['종목코드'].values):
    streamlit.loc[streamlit['종목코드'] == code, '시초/공모(%)'] = df.loc[i, '시초/공모(%)']

streamlit
# streamlit.to_csv(data_path + 'data_streamlit_df.csv',index=False)
# streamlit.to_csv(save_to + 'data_streamlit_df.csv',index=False)

,기업명,예측일,공모희망가(원),공모가(원),주간사,종목코드,신규상장일,공모주일정,시초/공모(%),model_score,f_score,t_score,a_score,범주
0,시지트로닉스,2023.07.18,"18,000~20,000","25,000",유안타증권,429270,2023/08/03,2023.07.24~07.25,5.4%,61.333207,44.493934,83.923214,88.000684,3
1,엠아이큐브솔루션,2023.07.20,"8,500~10,000","12,000",한국투자증권,373170,2023/08/04,2023.07.26~07.27,207.5%,74.163619,40.514965,19.771053,78.897444,2
2,파두,2023.07.24,"26,000~31,000","31,000","NH투자증권,한국투자증권,한화증권,현대차증권,유진투자증권,KB증권",440110,2023/08/07,2023.07.27~07.28,-15.16%,22.866463,49.079893,59.542354,36.982670,0
3,코츠테크놀로지,2023.07.26,"10,000~11,500","13,000","한국투자증권,하이투자증권",448710,2023/08/10,2023.08.01~08.02,92.31%,75.684666,44.091512,90.022065,86.333567,2
4,큐리옥스바이오시스템즈,2023.07.27,"13,000~16,000","13,000",키움증권,445680,2023/08/10,2023.08.01~08.02,0%,27.494307,42.813163,41.677069,17.898831,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,대한과학,2011.09.22,"2,400~3,200","3,000",대우증권,131220,2011/10/11,2011.09.29~09.30,67.0,60.798895,56.465009,65.616774,47.048076,2
759,넥솔론(유가),2011.09.28,"6,700~8,000","4,000",우리투자증권,110570,2011/10/14,2011.10.05~10.06,50.0,30.162639,21.383544,31.581769,2.316991,1
760,제닉,2011.07.18,"19,000~22,000","22,000",교보증권,123330,2011/08/03,2011.07.25~07.26,61.82,54.892246,49.192355,75.272435,48.457093,2
761,아이씨디,2011.07.18,"23,000~27,000","34,000",동양증권,040910,2011/08/05,2011.07.26~07.27,84.71,45.590668,33.732263,61.857874,46.824057,2
